### Import Packages

In [ ]:
from keras.utils import to_categorical
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from matplotlib import image
from matplotlib import pyplot
import pandas as pd
from keras.utils import to_categorical
from keras import layers
from keras import models
from keras import regularizers
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import matplotlib.pyplot as plt


### Load Test Dataframe

In [2]:
df = pd.read_csv('Test.csv')

### Generate Images From Directory

In [3]:
# Path Directory
train_data_dir = '../Mod 6 Project/Data/Train'
test_data_dir = '../Mod 6 Project/Data/Test'

In [4]:
# Train batches
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
                    train_data_dir,
                    target_size=(64,64),
                    classes=[str(i) for i in np.arange(43)],
                    batch_size=39209)
# Test batches
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
                    test_data_dir,
                    target_size=(64,64),
                    classes=[],
                    batch_size=12630)

Found 39209 images belonging to 43 classes.
Found 12630 images belonging to 1 classes.


In [5]:
# Extract train/test batch sequences
train_data, train_labels = next(train_batches)
test_data, test_labels = next(test_batches)

In [6]:
print('test data: {}'.format(test_data.shape))
print('test label: {}'.format(test_labels.shape))
print('train data: {}'.format(train_data.shape))
print('train label: {}'.format(train_labels.shape))

test data: (12630, 64, 64, 3)
test label: (12630, 1)
train data: (39209, 64, 64, 3)
train label: (39209, 43)


## Reshaping Train and Test Labels from 1 to 43 Columns

In [7]:
# train labels
train_labels = np.array([list(img).index(1.0) for img in train_labels])

In [8]:
# test labels
path_test = [int(i.replace('Test/','').replace('.png','')) for i in list(df['Path'])] # remove 'Test/','.png'
id_test = list(df['ClassId'])
path_id = dict(zip(path_test, id_test)) # tuple of path, id
labels = [path_id[batch] for batch in list(test_batches.index_array)]
test_labels = np.array(labels)

In [9]:
print('test label: {}'.format(test_labels.shape))
print('train label: {}'.format(train_labels.shape))

test label: (12630,)
train label: (39209,)


In [10]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [11]:
print('test label: {}'.format(test_labels.shape))
print('train label: {}'.format(train_labels.shape))

test label: (12630, 43)
train label: (39209, 43)


### C-Neural Network Architecture - RMSprop

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3))) # ADDED EPOCH
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),padding='same', activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(43, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

x_val = train_data[:3000]
partial_x_train = train_data[3000:]

y_val = train_labels[:3000]
partial_y_train = train_labels[3000:]

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=250,
                    validation_data=(x_val, y_val))
model.save('RMSPROP_FINAL.h5')

Train on 36209 samples, validate on 3000 samples
Epoch 1/20
36209/36209 [==============================] - 167s 5ms/step - loss: 2.0183 - accuracy: 0.4617 - val_loss: 1.3582 - val_accuracy: 0.7443
Epoch 2/20
36209/36209 [==============================] - 165s 5ms/step - loss: 0.5220 - accuracy: 0.8440 - val_loss: 0.5402 - val_accuracy: 0.9537
Epoch 3/20
36209/36209 [==============================] - 154s 4ms/step - loss: 0.2096 - accuracy: 0.9386 - val_loss: 0.3902 - val_accuracy: 0.9663
Epoch 4/20
36209/36209 [==============================] - 157s 4ms/step - loss: 0.1229 - accuracy: 0.9647 - val_loss: 0.1897 - val_accuracy: 0.9753
Epoch 5/20
22750/36209 [=================>............] - ETA: 57s - loss: 0.0855 - accuracy: 0.9756

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy: {}'.format(test_acc))

In [ ]:


predictions = model.predict(test_data)
predictions = np.around(predictions)
f1_score = f1_score(test_labels, predictions, average='micro')
print('f1 score: {}'.format(f1_score))

In [ ]:

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : RMSprop', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

### C-Neural Network Architecture - Adam

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3))) # ADDED EPOCH
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),padding='same', activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(43, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Validation 

In [ ]:
x_val = train_data[:3000]
partial_x_train = train_data[3000:]

y_val = train_labels[:3000]
partial_y_train = train_labels[3000:]

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=250,
                    validation_data=(x_val, y_val))

### Model Evaluations

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy: {}'.format(test_acc))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

predictions = model.predict(test_data)
predictions = np.around(predictions)
f1_score = f1_score(test_labels, predictions, average='micro')
print('f1 score: {}'.format(f1_score))

In [ ]:
import copy

test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
print(float(np.sum(np.array(test_labels) == np.array(test_labels_copy))) / len(test_labels))

In [ ]:
from keras.models import load_model
model.save('Adam_FINAL.h5')
# my_model = load_model('CNN_Drop_Bias_500BS+validation_padding.h5')

# my_prediction = my_model.predict(test_data)